In [2]:
# python version > 3.10 or 3.11 
!pip3 install --upgrade google-cloud-aiplatform
!pip3 install --upgrade google-cloud-bigquery
!pip3 install --upgrade google-cloud-storage
!pip3 install --upgrade google-auth
# if already exist please ignore the installations

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-5.29.2-cp310-abi3-win_amd64.whl.metadata (592 bytes)
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   - -------------------------------------- 0.2/6.9 MB 7.3 MB/s eta 0:00:01
   --- ------------------------------------ 0.6/6.9 MB 6.6 MB/s eta 0:00:01
   ----- ---------------------------------- 0.9/6.9 MB 7.2 MB/s eta 0:00:01
   ----- ---------------------------------- 0.9/6.9 MB 7.2 MB/s eta 0:00:01
   ----- ---------------------------------- 0.9/6.9 MB 4.2 MB/s eta 0:00:02
   ---------- ----------------------------- 1.9/6.9 MB 6.6 MB/s eta 0:00:01
   ----------- ---------------------------- 1.9/6.9 MB 6.9 MB/s eta 0:00:01
   ----------- ---------------------------- 1.9/6.9 MB 6.9 MB/s eta 0:00:01
   ---------------- ----------------------- 2.8/6.9 MB 6.8 MB/s eta 0:00:01
   ------------------ --------------------- 3.1/6.9 MB 6.8 MB/s eta 0:00:01
   -------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.2 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform


## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [8]:
project_id = 'dtidsus'
dataset_id = 'capstone'
table_id = 'data_bike_sharing'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'dharmawan/data_bike_sharing.csv'
client = bigquery.Client(project='dtidsus')

DefaultCredentialsError: File trial_bigq.json was not found.

In [11]:
try : 
    storage_client = storage.Client(project='dtidsus')
    bucket = storage_client.get_bucket(bucket_name)
    data_capstone = bucket.blob('dharmawan/data_bike_sharing.csv')
    data_capstone.upload_from_filename(r'C:\Users\Dharma\Desktop\Capstone 3-4\data_bike_sharing.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [12]:
# Upload to Google Cloud Platform

try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('dharmawan/model/casual.pkl')
    blob_model.upload_from_filename('casual.pkl')

    print("Uploading model succeeded")
except Exception as e:
    print("An exception occurred while uploading the model:", e)

Uploading model succeeded


In [13]:
from google.cloud import aiplatform

In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [15]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='jaya_model_000',
    artifact_uri="gs://modul4/fine/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/6781529334889840640/operations/6411212066308227072
Model created. Resource name: projects/41965541199/locations/us-central1/models/6781529334889840640@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/6781529334889840640@1')


In [16]:
endpoint = aiplatform.Endpoint.create(
    display_name="fine-endpoint-000",
    project='dtidsus',
    location='us-central1',
)

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/1834698978176794624/operations/1083453707128930304
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/1834698978176794624
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/1834698978176794624')


In [17]:
min_replica_count: int = 1
max_replica_count: int = 1

In [ ]:
endpoint.deploy( 
    model=model,
    deployed_model_display_name='fine_model_000',
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/6781529334889840640 to Endpoint : projects/41965541199/locations/us-central1/endpoints/1834698978176794624
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/1834698978176794624/operations/5379887751640383488


In [ ]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/2241324766407426048"
REGION = 'us-central1'



In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)